# Goal
___
The goal of this week's HW is to generate a call/put value table under the concept of Binary Option Price Method. Such as the example below:
<img src="https://drive.google.com/uc?export=view&id=1c4Nn7m27eRBjF7PiCMcgc0hWf3vZFv26"  width="600" height="600">

# Presentation form
---
I present the result with three tables. One is for stock price under different periods. Another is for call value under different periods. The other is put value under different periods. Such as the picture below:<br />
![](https://drive.google.com/uc?export=view&id=1AQXm5WnumRjWp9uFiCIM1Zg5PJR6WAdi)

# Overall work flow
***
To generate the call/put value, my work flow can be seperared into 3 sections as below.<br />
## Step1 
請使用者輸入資料，並根據使用者想要計算的期數創造相對應的空矩陣(方便後續儲存計算出來的數值)。
## Step2 
根據使用者輸入的資料，從第一期開始往後推每一期之不同情況下的股價，直到最後一期。
## Step3 
根據計算出來最後一期所有可能的股價，可以計算每一種可能性下的報酬，也就是這支選擇權在最後一期時產生的價值，往回推每一期的現值直到第一期


# Present step by step
___
## Step1
Ask users to input the Stock Price, folds that stock will increase per rise, folds that stock will decrease per decline, the continuously compounded riskless rate per period, the strike price, and the number of periods.


In [21]:
import pandas as pd
import numpy as np
import math

In [47]:
print("What is the Stock Price?")
S = float(input())
print("How many folds the stock will increase per peirod if it goes up?")
u = float(input())
print("How many folds the stock will decrease per period if it goes down?")
d = float(input())
print("What is the continuously compounded riskless rate per period? (% p.a.)")
r = float(input())/100
print("What is the strike price")
X = float(input())
print("How many periods you would like to consider?")
n = int(input())
R=math.exp(r)
p=(R-d)/(u-d)

What is the Stock Price?
160
How many folds the stock will increase per peirod if it goes up?
1.5
How many folds the stock will decrease per period if it goes down?
0.5
What is the continuously compounded riskless rate per period? (% p.a.)
18.232
What is the strike price
150
How many periods you would like to consider?
3


In [1]:
# assume the user input Current Bond Price = 1029, Bond Par Value = 1000, Bond Coupon Rate = 5%, Years to Maturity=1.5, and frequency of payment per year=2.
import pandas as pd
import math

S=160 
u=1.5
d=0.5
r=18.232/100
X=150
n=3
R=math.exp(r)
p=(R-d)/(u-d)

## Step 2
根據使用者輸入的資料，從第一期開始往後推每一期之不同情況下的**股價**，直到最後一期。

In [28]:
# produce row index for the dataframe
row_name = list(range(int(n)+1))
row_name = [x for x in row_name]
row_name = ['第'+ str(x) +'期' for x in row_name]
row_name

['第0期', '第1期', '第2期', '第3期']

In [29]:
# produce row index for the dataframe
col_name = list(range(int(n)+1))
col_name = [x for x in col_name]
col_name = ['下跌'+ str(x) +'次' for x in col_name]
col_name

['下跌0次', '下跌1次', '下跌2次', '下跌3次']

In [30]:
df_StockPrice = pd.DataFrame(index=row_name, columns=col_name)
df_StockPrice = df_StockPrice.fillna(0) # with 0s rather than NaNs
df_StockPrice

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,0,0,0,0
第1期,0,0,0,0
第2期,0,0,0,0
第3期,0,0,0,0


In [61]:
for i in list(range(len(row_name))):
    for j in list(range(len(row_name))):
        if i<j:
            df_StockPrice.iloc[i,j] = str('NA')
        else:
            df_StockPrice.iloc[i,j] = S*u**(i-j)*d**j
df_StockPrice

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,160.0,NA,NA,NA
第1期,240.0,80,NA,NA
第2期,360.0,120,40,NA
第3期,540.0,180,60,20


## Step 3
根據前面每一期計算出來的股價，計算該期**Call的價值(價格)**

In [11]:
# produce row index for the dataframe
row_name = list(range(int(n)+1))
row_name = [x for x in row_name]
row_name = ['第'+ str(x) +'期' for x in row_name]
row_name

['第0期', '第1期', '第2期', '第3期']

In [12]:
# produce row index for the dataframe
col_name = list(range(int(n)+1))
col_name = [x for x in col_name]
col_name = ['下跌'+ str(x) +'次' for x in col_name]
col_name

['下跌0次', '下跌1次', '下跌2次', '下跌3次']

In [19]:
import pandas as pd
df_call = pd.DataFrame(index=row_name, columns=col_name)
df_call = df_call.fillna(0) # with 0s rather than NaNs
df_call

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,0,0,0,0
第1期,0,0,0,0
第2期,0,0,0,0
第3期,0,0,0,0


In [63]:
# 計算Call的價格
for i in reversed(list(range(len(row_name)))):
    for j in reversed(list(range(len(row_name)))):
        if i<j:
            df_call.iloc[i,j] = str('NA')
        else:
            if i == len(row_name)-1:
                if (S*u**(i-j)*d**j)>X:
                    df_call.iloc[i,j] = (S*u**(i-j)*d**j)-X
                else:
                    df_call.iloc[i,j] = 0
            else:
                df_call.iloc[i,j] = (p*df_call.iloc[i+1,j] + (1-p)*df_call.iloc[i+1,j+1])/R

df_call

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,85.0692,NA,NA,NA
第1期,141.458,10.2083,NA,NA
第2期,235,17.5,0,NA
第3期,390,30,0,0


## Step 4
根據前面每一期計算出來的股價，計算該期**Put的價值(價格)**

In [23]:
# produce row index for the dataframe
row_name = list(range(int(n)+1))
row_name = [x for x in row_name]
row_name = ['第'+ str(x) +'期' for x in row_name]
row_name

['第0期', '第1期', '第2期', '第3期']

In [24]:
# produce row index for the dataframe
col_name = list(range(int(n)+1))
col_name = [x for x in col_name]
col_name = ['下跌'+ str(x) +'次' for x in col_name]
col_name

['下跌0次', '下跌1次', '下跌2次', '下跌3次']

In [25]:
import pandas as pd
df_put = pd.DataFrame(index=row_name, columns=col_name)
df_put = df_put.fillna(0) # with 0s rather than NaNs
df_put

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,0,0,0,0
第1期,0,0,0,0
第2期,0,0,0,0
第3期,0,0,0,0


In [69]:
# 計算Put的價格
for i in reversed(list(range(len(row_name)))):
    for j in reversed(list(range(len(row_name)))):
        if i<j:
            df_put.iloc[i,j] = str('NA')
        else:
            if i == len(row_name)-1:
                if (S*u**(i-j)*d**j)<X:
                    df_put.iloc[i,j] = X-(S*u**(i-j)*d**j)
                else:
                    df_put.iloc[i,j] = 0
            else:
                df_put.iloc[i,j] = (p*df_put.iloc[i+1,j] + (1-p)*df_put.iloc[i+1,j+1])/R

df_put = df_put.round(4)
df_put

,下跌0次,下跌1次,下跌2次,下跌3次
第0期,11.8752,NA,NA,NA
第1期,5.6251,34.3753,NA,NA
第2期,0.0000,22.5002,85.0002,NA
第3期,0.0000,0,90,130


In [70]:
dfs = [df_StockPrice, df_call, df_put]

df_combined = pd.concat(
    [df.rename(columns=lambda x: x.zfill(4)) for df in dfs],
    keys=['Stock Price','Call Value', 'Put Value'],
    axis=1
)

df_combined

Stock Price                Call Value                    Put Value  \
           下跌0次 下跌1次 下跌2次 下跌3次       下跌0次     下跌1次 下跌2次 下跌3次      下跌0次   
第0期       160.0   NA   NA   NA    85.0692       NA   NA   NA   11.8752   
第1期       240.0   80   NA   NA    141.458  10.2083   NA   NA    5.6251   
第2期       360.0  120   40   NA        235     17.5    0   NA    0.0000   
第3期       540.0  180   60   20        390       30    0    0    0.0000   

                            
        下跌1次     下跌2次 下跌3次  
第0期       NA       NA   NA  
第1期  34.3753       NA   NA  
第2期  22.5002  85.0002   NA  
第3期        0       90  130